In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Step 1: Load and clean data from Excel

In [ ]:

file_path = "../data/obrot_nieruchomosciami_w_2023_r_003.xlsx"

def load_and_clean_table(filepath, sheet_name, skiprows=4):
    df = pd.read_excel(filepath, sheet_name=sheet_name, skiprows=skiprows)
    df.columns = ['Category', 'Number of Properties', 'Value (thousands PLN)', 'Usable Area (m2)']
    df = df.dropna(subset=['Value (thousands PLN)', 'Usable Area (m2)'])
    df['Value (thousands PLN)'] = pd.to_numeric(df['Value (thousands PLN)'], errors='coerce')
    df['Usable Area (m2)'] = pd.to_numeric(df['Usable Area (m2)'], errors='coerce')
    return df

def calculate_avg_price_per_m2(df):
    total_value_pln = df['Value (thousands PLN)'].sum() * 1000
    total_area = df['Usable Area (m2)'].sum()
    return total_value_pln / total_area


In [ ]:
# Load data for primary and secondary markets
df_primary = load_and_clean_table(file_path, "Tabl_8")
df_secondary = load_and_clean_table(file_path, "Tabl_10")

avg_price_primary = calculate_avg_price_per_m2(df_primary)
avg_price_secondary = calculate_avg_price_per_m2(df_secondary)

print(f"Primary market avg price: {avg_price_primary:.2f} PLN/m²")
print(f"Secondary market avg price: {avg_price_secondary:.2f} PLN/m²")

In [ ]:
# Common simulation parameters
mu = 0.03
sigma = 0.15
T = 3
n_simulations = 10000
n_steps = 12 * T

In [ ]:
def monte_carlo_simulation(initial_price):
    dt = T / n_steps
    prices = np.zeros((n_simulations, n_steps + 1))
    prices[:, 0] = initial_price

    for t in range(1, n_steps + 1):
        Z = np.random.standard_normal(n_simulations)
        prices[:, t] = prices[:, t - 1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)

    final_prices = prices[:, -1]
    mean_price = np.mean(final_prices)

    # Histogram
    plt.hist(final_prices, bins=50, color='skyblue', edgecolor='black')
    plt.axvline(mean_price, color='red', linestyle='--', label=f'Mean: {mean_price:.2f} PLN/m²')
    plt.title(f"Monte Carlo Simulation\nForecast for Initial Price {initial_price:.2f} PLN/m²")
    plt.xlabel("Price per m² [PLN]")
    plt.ylabel("Number of Simulations")
    plt.legend()
    plt.grid(True)
    plt.show()

    return mean_price

In [ ]:

mc_primary = monte_carlo_simulation(avg_price_primary)
mc_secondary = monte_carlo_simulation(avg_price_secondary)
print(f"Monte Carlo forecast (primary): {mc_primary:.2f} PLN/m²")
print(f"Monte Carlo forecast (secondary): {mc_secondary:.2f} PLN/m²")

In [ ]:
# MLMC Simulation
def simulate_gbm(level, n_samples, T, initial_price, mu, sigma):
    n_steps = 2 ** level
    dt = T / n_steps
    prices = np.full(n_samples, initial_price)
    for _ in range(n_steps):
        Z = np.random.standard_normal(n_samples)
        prices *= np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    return prices

In [ ]:
def mlmc_estimate(max_level, samples_per_level, T, initial_price, mu, sigma):
    estimate = 0.0
    for level in range(max_level + 1):
        n = samples_per_level[level]
        if level == 0:
            f_coarse = simulate_gbm(0, n, T, initial_price, mu, sigma)
            estimate += np.mean(f_coarse)
        else:
            f_fine = simulate_gbm(level, n, T, initial_price, mu, sigma)
            f_coarse = simulate_gbm(level - 1, n, T, initial_price, mu, sigma)
            diff = f_fine - f_coarse
            estimate += np.mean(diff)
    return estimate

In [ ]:
max_level = 4
samples_per_level = [10000, 5000, 2500, 1250, 625]

mlmc_primary = mlmc_estimate(max_level, samples_per_level, T, avg_price_primary, mu, sigma)
mlmc_secondary = mlmc_estimate(max_level, samples_per_level, T, avg_price_secondary, mu, sigma)

print(f"MLMC forecast (primary): {mlmc_primary:.2f} PLN/m²")
print(f"MLMC forecast (secondary): {mlmc_secondary:.2f} PLN/m²")

In [ ]:
# Comparison bar chart
labels = ['Primary', 'Secondary']
mc_values = [mc_primary, mc_secondary]
mlmc_values = [mlmc_primary, mlmc_secondary]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
bars1 = ax.bar(x - width/2, mc_values, width, label='Monte Carlo', color='skyblue')
bars2 = ax.bar(x + width/2, mlmc_values, width, label='MLMC', color='lightgreen')

ax.set_ylabel('Forecasted Price (PLN/m²)')
ax.set_title('Forecasted Prices by Method and Market')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


In [ ]:
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.grid(True)
plt.show()
